In [1]:
from pyspark.sql import SparkSession

In [2]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.13:4.0.0",
        "org.apache.hadoop:hadoop-aws:3.4.0",
    ]

    def __init__(self, access_key, secret_key, endpoint):
        self._access_key = access_key
        self._secret_key = secret_key
        self._endpoint = endpoint
        

    def start(
        self,
        app_name: str = "Airflow Spark Delta Minio App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[*]",
    ):

        builder = (
            SparkSession
            .Builder()
            .appName(app_name)
            .config("spark.hadoop.fs.s3a.access.key", self._access_key)
            .config("spark.hadoop.fs.s3a.secret.key", self._secret_key)
            .config("spark.hadoop.fs.s3a.endpoint", self._endpoint)
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
            .config("spark.hadoop.fs.s3a.path.style.access", "true")
            #
            .config("spark.hadoop.delta.enableFastS3AListFrom", "true")
            #
            # .config("spark.hadoop.hive.metastore.uris", "thrift://127.0.0.1:9083")
            #
            .config("spark.executor.memory", executor_memory)
            #
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            #
            # .config("spark.sql.warehouse.dir", "s3a://lakehouse")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            # .config("spark.sql.hive.metastore.version", "4.0.0")
            # .config("spark.sql.hive.metastore.jars", "lib/*")
            #
            .config("spark.jars.packages", ",".join(self.packages))
            #
            .master(master_url)
            # .enableHiveSupport()
        )

        return builder.getOrCreate()

In [3]:
spark = LazySparkSession(
    access_key="31aa474f9f2509cf", 
    secret_key="nIYfEB7Vjvly5sfH2JU6WJGrinawA+hHSoHlyinEe74=", 
    endpoint="http://host.docker.internal:10000"
).start()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/22 07:19:32 WARN Utils: Your hostname, PC21-107632, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/22 07:19:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/elicacio/Workspace-personal/projeto-lakehouse/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/elicacio/.ivy2.5.2/cache
The jars for the packages stored in: /home/elicacio/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-de7f38d6-e4c1-408f-a4d2-c7654574680b;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 i

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 41180)
Traceback (most recent call last):
  File "/usr/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/home/elicacio/Workspace-personal/projeto-lakehouse/.venv/lib/python3.12/site-packages/pyspark/accumulators.py", line 299, in handle
    poll(accum_updates)
  File "/home/elicacio/Workspace-personal/projeto-lakehouse/.venv/lib/python3.12/site-packages/pyspark/accumulators.py", line 271, in poll
    if self.rfile in r and func():
                           ^^^^^^


In [4]:
df = spark.createDataFrame([(1,2), (1,2), (1,2)], schema="a integer, b integer")

In [6]:
df.write.format("delta").mode('overwrite').save("s3a://bronze/numbers/integers")

25/08/22 07:20:14 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
25/08/22 07:20:19 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to numbers/integers/part-00000-83a9f518-b421-400e-8bd5-dc88769703d7-c000.snappy.parquet. This is Unsupported
25/08/22 07:20:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
spark.read.format('delta').load('s3a://bronze/numbers/integers').show()

+---+---+
|  a|  b|
+---+---+
|  1|  2|
|  1|  2|
|  1|  2|
+---+---+



25/08/22 07:34:38 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 125090 ms exceeds timeout 120000 ms
25/08/22 07:34:38 WARN SparkContext: Killing executors is not supported by current scheduler.
25/08/22 07:32:43 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at o

In [ ]:
25/08/21 07:54:12 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.

25/08/21 07:54:15 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
Hive Session ID = e38db77a-6afe-437e-bc8e-d5fa7f632b38
25/08/21 07:54:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/08/21 07:54:51 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to numbers/integer/part-00000-8c5f40d3-4def-4580-b432-fc63efb209cc-c000.snappy.parquet. This is Unsupported
25/08/21 07:55:02 WARN HiveExternalCatalog: Could not alter schema of table `numbers`.`integer` in a Hive compatible way. Updating Hive metastore in Spark SQL specific format.

Caused by: org.apache.spark.rpc.RpcEndpointNotFoundException: Cannot find endpoint: spark://CoarseGrainedScheduler@10.255.255.254:34253

WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic